<a href="https://colab.research.google.com/github/sashkoangelov/NLP_final_project/blob/main/model2_backtranslation_overlap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs and imports

In [1]:
import torch
import random
import transformers
import copy
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering, TrainingArguments, Trainer, AdamW
from torch.utils.data import Dataset, DataLoader
import os
import json
from pathlib import Path
from tqdm import tqdm

In [2]:
print(transformers.__version__)

4.38.2


#Data loading

In [3]:
def read_data(path):
    with open(path, 'r', encoding='utf-8') as f:  # Use 'r' for reading text files, and specify encoding
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []

    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']

                # Check if 'answers' is present and has entries
                if 'answers' in qa and qa['answers']:
                    for answer in qa['answers']:
                        contexts.append(context)
                        questions.append(question)
                        answers.append(answer)
                else:
                    # Handle unanswerable questions
                    contexts.append(context)
                    questions.append(question)
                    answers.append({"text": "", "answer_start": -1})

    return contexts, questions, answers

In [4]:
train_contexts, train_questions, train_answers = read_data('/content/drive/MyDrive/NLP/train-v2.0.json')
bt_contexts, bt_questions, bt_answers = read_data('/content/drive/MyDrive/NLP/train-v2.0-with-back-translation-fr.json')
val_contexts, val_questions, val_answers = read_data('/content/drive/MyDrive/NLP/dev-v2.0.json')

# Data visualization

## Displaying random examples

In [5]:
def display_random_examples(contexts, questions, answers, num_examples=3):

    for _ in range(num_examples):
        rand = random.randint(0, len(contexts) - 1)
        context = contexts[rand]
        question = questions[rand]

        answer_text = answers[rand]['text'] if answers[rand]['text'] != '' else 'No answer'
        answer_start = answers[rand]['answer_start'] if answers[rand]['answer_start'] != '' else -1

        print(f"Question: {question}\n")
        print(f"Answer: {answer_text}")
        print(f"Answer Starts at: {answer_start}\n")

        # Highlighting answer in context
        if answer_start != -1:
            highlighted_context = context[:answer_start] + "\033[1;31m" + context[answer_start:answer_start+len(answer_text)] + "\033[0m" + context[answer_start+len(answer_text):]
        else:
            highlighted_context = context

        # Wrapping context for readability
        wrapped_context = "\n".join([highlighted_context[i:i+80] for i in range(0, len(highlighted_context), 80)])
        print(f"Context:\n{wrapped_context}\n")
        print("-"*80)


In [6]:
display_random_examples(train_contexts, train_questions, train_answers, 3)

Question: What did the Easter and Western Catholic church have in common?

Answer: No answer
Answer Starts at: -1

Context:
The canon law of the Eastern Catholic Churches, which had developed some differe
nt disciplines and practices, underwent its own process of codification, resulti
ng in the Code of Canons of the Eastern Churches promulgated in 1990 by Pope Joh
n Paul II.

--------------------------------------------------------------------------------
Question: How do Palermo's temperatures compare to the rest of Europe?

Answer: one of the warmest cities in Europe
Answer Starts at: 213

Context:
Palermo experiences a hot-summer Mediterranean climate (Köppen climate classific
ation: Csa). Winters are cool and wet, while summers are hot and dry. Temperatur
es in autumn and spring are usually mild. Palermo is one of the warmest c
ities in Europe (mainly due to its warm nights), with an average annual air 
temperature of 18.5 °C (65.3 °F). It receives approximately 2,530 hours of suns

## Display a single example

In [7]:
def display_example(contexts, questions, answers, example):

    rand = example
    context = contexts[rand]
    question = questions[rand]

    answer_text = answers[rand]['text'] if answers[rand]['text'] != '' else 'No answer'
    answer_start = answers[rand]['answer_start'] if answers[rand]['answer_start'] != '' else -1

    print(f"Question: {question}\n")
    print(f"Answer: {answer_text}")
    print(f"Answer Starts at: {answer_start}\n")

    # Highlighting answer in context
    if answer_start != -1:
        highlighted_context = context[:answer_start] + "\033[1;31m" + context[answer_start:answer_start+len(answer_text)] + "\033[0m" + context[answer_start+len(answer_text):]
    else:
        highlighted_context = context

    # Wrapping context for readability
    wrapped_context = "\n".join([highlighted_context[i:i+80] for i in range(0, len(highlighted_context), 80)])
    print(f"Context:\n{wrapped_context}\n")
    print("-"*80)

In [8]:
display_example(train_contexts, train_questions, train_answers, 19)

Question: When did Beyonce take a hiatus in her career and take control of her management?

Answer: 2010
Answer Starts at: 586

Context:
Following the disbandment of Destiny's Child in June 2005, she released her seco
nd solo album, B'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", an
d "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nomin
ated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2
006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta Jam
es in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (
2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-set
ting six Grammy Awards in 2010, including Song of the Year for "Singl
e Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took 
over management of her career; her fourth album 4 (2011) was subsequently mellow
er in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her

In [9]:
display_example(bt_contexts, bt_questions, bt_answers, 19)

Question: When did Beyonce take a break in her career and take control of her leadership?

Answer: 2010
Answer Starts at: 586

Context:
After the dismantling of Destiny's Child in June 2005, she released her second s
olo album, B'Day (2006), which contains hits "Déjà Vu", "Irreplaceable" and "Bea
utiful Liar". Beyoncé also ventured into the performance, with a nominated Golde
n Globe performance in Dreamgirls (2006), and featured roles in The Pink Panther
 (2006) and Obsessed (2009). Her marriage to rapper Jay Z and the performance of
 Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha
 Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and obtained a
 record of six Grammy Awards in 2010, including Song of the Year for 
"Sing of the Single Ladies (Put a Ring on It)". Beyoncé took a hiatus of music i
n 2010 and took over the direction of her career; her fourth album 4 (2011) was 
then melliferous in tone, exploring the 1970s pop, and 1990s soul

In [14]:
display_example(updated_contexts, updated_questions, updated_answers, 18)

Question: When did Beyonce take a hiatus in her career and take control of her management?

Answer: 2010
Answer Starts at: 592

Context:
After the dismantling of Destiny's Child in June 2005, she released her second s
olo album, B'Day (2006), which contains hits "Déjà Vu", "Irreplaceable" and "Bea
utiful Liar". Beyoncé also ventured into the performance, with a nominated Golde
n Globe performance in Dreamgirls (2006), and featured roles in The Pink Panther
 (2006) and Obsessed (2009). Her marriage to rapper Jay Z and the performance of
 Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha
 Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and obtained a
 record of six Grammy Awards in 2010, including Song of the Year for 
"Sing of the Single Ladies (Put a Ring on It)". Beyoncé took a hiatus of music i
n 2010 and took over the direction of her career; her fourth album 4 (2011) was 
then melliferous in tone, exploring the 1970s pop, and 1990s sou

# Assign new start positions for the answers in the back translated contexts if it exists

In [11]:
def assign_new_start_pos(bt_contexts, train_questions, train_answers):
    updated_contexts = []
    updated_questions = []
    updated_answers = []

    for context, question, answer in zip(bt_contexts, train_questions, train_answers):
        try:

            # Attempt to find the non-translated answer in the translated context
            new_start = context.index(answer['text'])

            # If found, update the entry
            updated_contexts.append(context)
            updated_questions.append(question)
            updated_answers.append({'text': answer['text'], 'answer_start': new_start})

        except ValueError:
            continue

    return updated_contexts, updated_questions, updated_answers

In [12]:
updated_contexts, updated_questions, updated_answers = assign_new_start_pos(bt_contexts, train_questions, train_answers)

In [13]:
print("Preserved samples:", round((len(updated_contexts) / len(bt_contexts))*100, 2), "%")

Preserved samples: 77.31 %


## Discarding identical triplets of contexts, questions and answers

In [15]:
def discard_duplicate_triplets(train_contexts, train_questions, train_answers, updated_contexts, updated_questions, updated_answers):
    seen = set()
    unique_triplets = []

    # Combine the training and updated datasets into one list of triplets
    combined_triplets = list(zip(train_contexts, train_questions, train_answers)) + list(zip(updated_contexts, updated_questions, updated_answers))

    for context, question, answer in combined_triplets:
        # Assuming 'answer' is a dict with a 'text' key, or adjust according to your structure
        triplet_tuple = (context, question, answer['text'])  # Convert to a tuple for hashability

        if triplet_tuple not in seen:
            seen.add(triplet_tuple)
            # Append the original items, not the tuple
            unique_triplets.append((context, question, answer))

    # Separate the combined list back into individual lists
    unique_contexts, unique_questions, unique_answers = zip(*unique_triplets)

    # Convert from tuples back to lists (if needed)
    return list(unique_contexts), list(unique_questions), list(unique_answers)


In [16]:
unique_contexts, unique_questions, unique_answers = discard_duplicate_triplets(train_contexts, train_questions, train_answers, updated_contexts, updated_questions, updated_answers)

In [17]:
print("Preserved samples:", round(((len(unique_contexts) - len(train_contexts)) / len(bt_contexts))*100, 2), "%")

Preserved samples: 77.21 %


# Data pre-processing

In [18]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

## Prepare features V2

In [19]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [20]:
def prepare_train_features(contexts, questions, answers):
    questions = [question.strip() for question in questions]

    tokenized = tokenizer(
        questions,
        contexts,
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
        return_tensors="pt"
    )

    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")

    tokenized["start_positions"] = []
    tokenized["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized["input_ids"][i]
        # Correctly find the CLS token index
        cls_index = (input_ids == tokenizer.cls_token_id).nonzero(as_tuple=True)[0][0].item()
        sequence_ids = tokenized.sequence_ids(i)
        sample_index = sample_mapping[i]
        answer = answers[sample_index]

        if answer["answer_start"] == -1:
            tokenized["start_positions"].append(cls_index)
            tokenized["end_positions"].append(cls_index)
        else:
            start_char = answer["answer_start"]
            end_char = start_char + len(answer["text"])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized["start_positions"].append(cls_index)
                tokenized["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized["start_positions"].append(token_start_index - 1)

                while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized["end_positions"].append(token_end_index + 1)

    return tokenized

In [21]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings, answers=None):
        self.encodings = encodings
        self.answers = answers

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [22]:
train_tokenized = prepare_train_features(unique_contexts, unique_questions, unique_answers)
train_dataset = QADataset(train_tokenized)

val_tokenized = prepare_train_features(val_contexts, val_questions, val_answers)
val_dataset = QADataset(val_tokenized)

#Fine-tuning

In [23]:
model = RobertaForQuestionAnswering.from_pretrained("roberta-base")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Number of epochs
n_epochs = 2

for epoch in range(n_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{n_epochs}'):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{n_epochs}, Training Loss: {avg_loss}')

    # Validation phase
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            total_val_loss += outputs.loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f'Epoch {epoch + 1}/{n_epochs}, Validation Loss: {avg_val_loss}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/2:   0%|          | 0/14731 [00:00<?, ?it/s]<ipython-input-21-b92ab51b90c2>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1/2: 100%|██████████| 14731/14731 [1:22:11<00:00,  2.99it/s]


Epoch 1/2, Training Loss: 1.0203670426635698
Epoch 1/2, Validation Loss: 1.1887628478816243


Epoch 2/2: 100%|██████████| 14731/14731 [1:22:12<00:00,  2.99it/s]


Epoch 2/2, Training Loss: 0.6869625579028377
Epoch 2/2, Validation Loss: 1.2309589987868104


In [25]:
directory_path = '/content/drive/MyDrive/NLP'
os.makedirs(directory_path, exist_ok=True)

In [26]:
model_path = os.path.join(directory_path, 'model2_backtranslation_overlapV2.pth')
torch.save(model, model_path)